## Real-TIme Multiclass Classification:

In [5]:
!pip install pyloudnorm

In [6]:
import pyloudnorm as pyln

filename = 'US8KS/engine_idling.wav'
data,rate = librosa.load(filename)
meter = pyln.Meter(rate) # create BS.1770 meter
loudness = meter.integrated_loudness(data) # measure loudness

In [7]:
loudness

-39.385357558920326

In [12]:
!pip install --upgrade pip setuptools wheel
!pip install -I tensorflow
!pip install -I keras

Requirement already up-to-date: wheel in c:\users\pedro\anaconda3\lib\site-packages (0.33.6)
  Found existing installation: pip 19.2.3
    Uninstalling pip-19.2.3:
      Successfully uninstalled pip-19.2.3


ERROR: Could not install packages due to an EnvironmentError: [WinError 5] Acesso negado: 'C:\\Users\\pedro\\AppData\\Local\\Temp\\pip-uninstall-hjx9yfo9\\pip.exe'
Consider using the `--user` option or check the permissions.



^C
  Using cached https://files.pythonhosted.org/packages/a1/6b/7f62017e3f0b32438dd90bdc1ff0b7b1448b6cb04a1ed84f37b6de95cd7b/h5py-2.10.0-cp37-cp37m-win_amd64.whl
  Using cached https://files.pythonhosted.org/packages/28/6a/8c1f62c37212d9fc441a7e26736df51ce6f0e38455816445471f10da4f0a/Keras_Preprocessing-1.1.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/61/51/046cbc61c7607e5ecead6ff1a9453fba5e7e47a5ea8d608cc7036586a5ef/scipy-1.4.1-cp37-cp37m-win_amd64.whl
  Using cached https://files.pythonhosted.org/packages/a9/38/f6d6d8635d496d6b4ed5d8ca4b9f193d0edc59999c3a63779cbc38aa650f/numpy-1.18.1-cp37-cp37m-win_amd64.whl
  Using cached https://files.pythonhosted.org/packages/71/e3/19762fdfc62877ae9102edf6342d71b28fbfd9dea3d2f96a882ce099b03f/Keras_Applications-1.0.8-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/65/eb/1f97cb97bfc2390a276969c6fae16075da282f5058082d4cb10c6c5c1dba/six-1.14.0-py2.py3-none-any.whl


ERROR: astroid 2.3.1 requires typed-ast<1.5,>=1.4.0; implementation_name == "cpython" and python_version < "3.8", which is not installed.
ERROR: tensorflow 1.15.0 has requirement tensorboard<1.16.0,>=1.15.0, but you'll have tensorboard 2.0.0 which is incompatible.
ERROR: astroid 2.3.1 has requirement six==1.12, but you'll have six 1.14.0 which is incompatible.
ERROR: Could not install packages due to an EnvironmentError: [WinError 5] Acesso negado: 'c:\\users\\pedro\\anaconda3\\Lib\\site-packages\\numpy\\core\\_multiarray_tests.cp37-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



In [28]:
#Load various imports
import pyaudio
import numpy as np
import librosa
import pylab
import datetime
import sys
import IPython.display as ipd
from IPython.display import clear_output
import plotly.graph_objs as go
import warnings
warnings.filterwarnings("ignore")
import joblib

#Choose the classification model (lr, svm, rf, knn, fnn)
model=joblib.load('fnn40.pkl')

RATE = 44100
INSTANCES = RATE * 4
CHUNK = int(RATE/20) # RATE / number of updates per second

def get_data(stream):
    data = np.fromstring(stream.read(CHUNK),dtype=np.int16) / 2**15
    return data

#Extract features from the audio stream (n_mfcc should be adjusted accordingly)
def extract_features_from_array(audio):
   
    try:
        mfccs = librosa.feature.mfcc(y=audio, sr=RATE, n_mfcc=40, hop_length=512)
        mfccsscaled = np.mean(mfccs.T,axis=0)
        
    except Exception as e:
        print("Error encountered while parsing file: ", file)
        return None 
     
    return mfccsscaled

# Select labels to detect
labels = ['Air Conditioner', 'Car Horn', 'Children Playing', 'Dog Bark',
       'Drilling', 'Engine Idling', 'Gunshot', 'Jackhammer', 'Siren',
       'Street Music']
selected_labels = ['Air Conditioner', 'Car Horn', 'Children Playing',
       'Drilling', 'Engine Idling', 'Gunshot', 'Jackhammer', 'Siren',
       'Street Music']

selected_indices = []
for selected_label in selected_labels:
    for index, label in enumerate(labels):
        if label == selected_label:
            selected_indices.append(index)

all_indices = [i for i in range(len(labels))]
not_selected_indices = [i for i in all_indices if i not in selected_indices]

if __name__=="__main__":
    
    if selected_labels != labels:
        selected_labels.append("Other")
    graph = [go.Bar(
            x=selected_labels,
            y=[0]*(len(selected_labels)+1))]
    layout = {'yaxis': {'range': [0, 1]}}
    fig = go.FigureWidget(graph, layout)
    bar = fig.data[0]
    ipd.display(fig)
    p=pyaudio.PyAudio()
    stream=p.open(format=pyaudio.paInt16,channels=1,rate=RATE,input=True,
                  frames_per_buffer=CHUNK)
    
    array = np.array([])
    for i in range(sys.maxsize**10):
        data = get_data(stream)
        meter = pyln.Meter(RATE) # create BS.1770 meter
        loudness_stream = meter.integrated_loudness(data) # measure loudness
        if loudness_stream > -70:
            if array.shape[0] < INSTANCES:
                array = np.append(array, data)
            else:
                array = array[nr_data.shape[0]:]
                array = np.append(array, data)
            if i % 1 == 0:
                features = extract_features_from_array(array)
                features = features.reshape(1, -1)
                predictions = np.array(model.predict_proba(features)[0])
                other_sum = np.sum(predictions[not_selected_indices])  if not_selected_indices else 0
                bar.y = np.append(predictions[selected_indices], other_sum)

            if i % 10 == 0:
                thresholds = np.argwhere(predictions[selected_indices] > 0.5)
                if thresholds:
                    with open("alarms.txt", "a") as file:
                        output_string = selected_labels[thresholds[0][0]] + " " + str(datetime.datetime.now()) + "\n"
                        file.write(output_string)
    stream.stop_stream()
    stream.close()
    p.terminate()

Using TensorFlow backend.


AttributeError: module 'tensorflow' has no attribute 'name_scope'

In [29]:
loudness_stream

NameError: name 'loudness_stream' is not defined

In [15]:
np.argwhere(predictions > 0.5)[0][0]

NameError: name 'predictions' is not defined

In [ ]:
predictions

In [16]:
noise_sample[np.isfinite(noise_sample)].shape

NameError: name 'noise_sample' is not defined

In [ ]:
noise_sample.shape

In [17]:
not_selected_indices

NameError: name 'not_selected_indices' is not defined

In [ ]:
ipd.Audio(data=array*200, rate=RATE)

In [18]:
CHUNK

NameError: name 'CHUNK' is not defined

In [ ]:
predictions

In [20]:
display_string = ""
for index, label in enumerate(labels):
                display_string += label + ": " + str(predictions[0][index]) + "\n"
            clear_output(wait=True)
            print(display_string[:-1])

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 4)

# Daqui para baixo, está a funcionar (com as samples)

In [21]:
# Load various imports
import numpy as np
#import pandas as pd
import os
import librosa
import ast

from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

from sklearn.metrics import confusion_matrix

from matplotlib import pyplot as plt
import seaborn as sn

from sklearn import metrics

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import Adam

Using TensorFlow backend.


AttributeError: module 'tensorflow' has no attribute 'name_scope'

### Função para extrair MFCCs de um ficheiro de áudio:

In [22]:
def extract_features(file_name):
   
    try:
        audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=20, hop_length=512)
        mfccsscaled = np.mean(mfccs.T,axis=0)
        
    except Exception as e:
        print("Error encountered while parsing file: ", file)
        return None 
     
    return mfccsscaled

In [23]:
file_name = 'US8KS/gun_shot.wav'
audio = extract_features(file_name)
audio = audio.reshape(1, -1)

loadedfnnpredictions=model.predict_proba(audio)
loadedfnnpredictions

NameError: name 'model' is not defined

In [ ]:
audio, sample_rate =  librosa.load(file_name, res_type='kaiser_fast') 

In [ ]:
np.max(audio)

In [ ]:
loadedfnn.predict_proba(file_name)

## Coisas do Chathuranga

In [24]:
import numpy as np
import librosa
import matplotlib.pyplot as plt
from keras.models import model_from_json
from sklearn.preprocessing import LabelEncoder
import IPython
import os
import pyaudio
import joblib
import noisereduce as nr

Using TensorFlow backend.


AttributeError: module 'tensorflow' has no attribute 'name_scope'

In [ ]:
!pip install noisereduce

In [25]:
#Load segment audio classification model
#model_path = r"best_model/"
#model_name = "audio_NN3_grouping2019_10_01_11_40_45_acc_91.28"
model=joblib.load('fnn40.pkl')

# Model reconstruction from JSON file
# with open(model_path + model_name + '.json', 'r') as f:
#     model = model_from_json(f.read())

# Load weights into the new model
#model.load_weights(model_path + model_name + '.h5')

# Replicate label encoder
lb = LabelEncoder()
lb.fit_transform(['Air Conditioner', 'Car Horn', 'Children Playing', 'Dog Bark',
       'Drilling', 'Engine Idling', 'Gunshot', 'Jackhammer', 'Siren',
       'Street Music'])

Using TensorFlow backend.


AttributeError: module 'tensorflow' has no attribute 'name_scope'

In [27]:
#Some Utils

# Plot audio with zoomed in y axis
def plotAudio(output):
    fig, ax = plt.subplots(nrows=1,ncols=1, figsize=(20,10))
    plt.plot(output, color='blue')
    ax.set_xlim((0, len(output)))
    ax.margins(2, -0.1)
    plt.show()

# Plot audio
def plotAudio2(output):
    fig, ax = plt.subplots(nrows=1,ncols=1, figsize=(20,4))
    plt.plot(output, color='blue')
    ax.set_xlim((0, len(output)))
    plt.show()

def minMaxNormalize(arr):
    mn = np.min(arr)
    mx = np.max(arr)
    return (arr-mn)/(mx-mn)

def predictSound(X):
    clip, index = librosa.effects.trim(X, top_db=20, frame_length=512, hop_length=64) # Empherically select top_db for every sample
    stfts = np.abs(librosa.stft(clip, n_fft=512, hop_length=256, win_length=512))
    stfts = np.mean(stfts,axis=1)
    stfts = minMaxNormalize(stfts)
    result = model.predict(np.array([stfts]))
    predictions = [np.argmax(y) for y in result]
    print(lb.inverse_transform([predictions[0]])[0])
    plotAudio2(clip)

In [26]:
CHUNKSIZE = 22050 # fixed chunk size
RATE = 22050

# initialize portaudio
p = pyaudio.PyAudio()
stream = p.open(format=pyaudio.paFloat32, channels=1, rate=RATE, input=True, frames_per_buffer=CHUNKSIZE)

#noise window
data = stream.read(10000)
noise_sample = np.frombuffer(data, dtype=np.float32)
print("Noise Sample")
plotAudio2(noise_sample)
loud_threshold = np.mean(np.abs(noise_sample)) * 10
print("Loud threshold", loud_threshold)
audio_buffer = []
near = 0

while(True):
    # Read chunk and load it into numpy array.
    data = stream.read(CHUNKSIZE)
    current_window = np.frombuffer(data, dtype=np.float32)
    
    #Reduce noise real-time
    current_window = nr.reduce_noise(audio_clip=current_window, noise_clip=noise_sample, verbose=False)
    
    if(audio_buffer==[]):
        audio_buffer = current_window
    else:
        if(np.mean(np.abs(current_window))<loud_threshold):
            print("Inside silence reign")
            if(near<10):
                audio_buffer = np.concatenate((audio_buffer,current_window))
                near += 1
            else:
                predictSound(np.array(audio_buffer))
                audio_buffer = []
                near
        else:
            print("Inside loud reign")
            near = 0
            audio_buffer = np.concatenate((audio_buffer,current_window))

# close stream
stream.stop_stream()
stream.close()
p.terminate()

Noise Sample


NameError: name 'plotAudio2' is not defined

In [ ]:
current_window